In [1]:
pip install pymupdf pytesseract Pillow langdetect

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install google-generativeai

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pymupdf easyocr langdetect Pillow

Note: you may need to restart the kernel to use updated packages.


In [4]:
import re
import fitz  # PyMuPDF
from langdetect import detect

# === CONFIGURATION ===
PDF_PATH = "C:/Users/OUMAIMA/Downloads/cv_oumeymaFinal (1).pdf"

def extract_text_from_pdf(pdf_path):
    """Extract text from PDF using PyMuPDF."""
    text = ""
    doc = fitz.open(pdf_path)
    for page in doc:
        page_text = page.get_text()
        if page_text.strip():
            text += page_text + "\n"
    doc.close()
    return text

def clean_text(text):
    """Remove special chars, bullet points, normalize spaces, and replace accented chars with non-accented equivalents."""
    # Replace accented characters with non-accented equivalents
    text = text.replace('à', 'a').replace('â', 'a').replace('ä', 'a')
    text = text.replace('é', 'e').replace('è', 'e').replace('ê', 'e').replace('ë', 'e')
    text = text.replace('î', 'i').replace('ï', 'i')
    text = text.replace('ô', 'o').replace('ö', 'o')
    text = text.replace('ù', 'u').replace('û', 'u').replace('ü', 'u')
    text = text.replace('ç', 'c')
    # Remove remaining special characters, keeping only alphanumeric and spaces
    text = re.sub(r'[^A-Za-z0-9\s]', ' ', text)
    # Normalize spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def detect_language(text):
    """Detect if the text is English or French."""
    try:
        lang = detect(text[:500])
        return "French" if lang == "fr" else "English" if lang == "en" else f"Other ({lang})"
    except:
        return "Unknown"

def extract_sections_regex(raw_text, language):
    """Extract Experience, Education, Skills, and Projects using regex, based on language, before cleaning."""
    sections = {
        "experience": "Non trouvée dans le CV",
        "education": "Non trouvée dans le CV",
        "skills": "Non trouvée dans le CV",
        "academic_projects": "Non trouvée dans le CV"
    }

    # Define patterns based on language, headers followed by \n or symbol (not space or .)
    if language == "French":
        exp_pattern = r'\b(exp[ée]rience|emploi|exp[ée]rience professionnelle)\b\s*([:;\-\n#|])(.*?)(?=\b(formation|[ée]ducation|comp[ée]tences|comp[ée]tences techniques|projets acad[ée]miques|projets)\b|$)'
        edu_pattern = r'\b(formation|[ée]ducation)\b\s*([:;\-\n#|])(.*?)(?=\b(comp[ée]tences|comp[ée]tences techniques|exp[ée]rience|emploi|exp[ée]rience professionnelle|projets acad[ée]miques|projets)\b|$)'
        skills_pattern = r'\b(comp[ée]tences|comp[ée]tences techniques)\b\s*([:;\-\n#|])(.*?)(?=\b(formation|[ée]ducation|exp[ée]rience|emploi|exp[ée]rience professionnelle|projets acad[ée]miques|projets)\b|$)'
        projects_pattern = r'\b(projets acad[ée]miques|projets)\b\s*([:;\-\n#|])(.*?)(?=\b(formation|[ée]ducation|comp[ée]tences|comp[ée]tences techniques|exp[ée]rience|emploi|exp[ée]rience professionnelle)\b|$)'
    else:  # English or Other
        exp_pattern = r'\b(experience|work history|professional experience)\b\s*([:;\-\n#|])(.*?)(?=\b(education|academic background|skills|technical skills|academic projects|projects)\b|$)'
        edu_pattern = r'\b(education|academic background)\b\s*([:;\-\n#|])(.*?)(?=\b(skills|technical skills|experience|work history|professional experience|academic projects|projects)\b|$)'
        skills_pattern = r'\b(skills|technical skills)\b\s*([:;\-\n#|])(.*?)(?=\b(education|academic background|experience|work history|professional experience|academic projects|projects)\b|$)'
        projects_pattern = r'\b(academic projects|projects)\b\s*([:;\-\n#|])(.*?)(?=\b(education|academic background|skills|technical skills|experience|work history|professional experience)\b|$)'

    # Match sections
    experience_match = re.search(exp_pattern, raw_text, re.IGNORECASE | re.DOTALL)
    education_match = re.search(edu_pattern, raw_text, re.IGNORECASE | re.DOTALL)
    skills_match = re.search(skills_pattern, raw_text, re.IGNORECASE | re.DOTALL)
    projects_match = re.search(projects_pattern, raw_text, re.IGNORECASE | re.DOTALL)

    # Assign matched content to sections and clean the extracted text
    if experience_match:
        sections["experience"] = clean_text(experience_match.group(3).strip())
    if education_match:
        sections["education"] = clean_text(education_match.group(3).strip())
    if skills_match:
        sections["skills"] = clean_text(skills_match.group(3).strip())
    if projects_match:
        sections["academic_projects"] = clean_text(projects_match.group(3).strip())

    return sections

# === MAIN ===
raw_text = extract_text_from_pdf(PDF_PATH)
language = detect_language(raw_text)  # Detect language on raw text

# Define section headers and not found message based on detected language
if language == "French":
    headers = {
        "experience": "EXPÉRIENCE",
        "education": "FORMATION",
        "skills": "COMPÉTENCES",
        "academic_projects": "PROJETS ACADÉMIQUES"
    }
    not_found = "Non trouvée dans le CV"
else:  # English or Other
    headers = {
        "experience": "EXPERIENCE",
        "education": "EDUCATION",
        "skills": "SKILLS",
        "academic_projects": "ACADEMIC PROJECTS"
    }
    not_found = "Not found in the CV"

# Extract sections using regex on raw text
sections = extract_sections_regex(raw_text, language)

# Update sections with language-appropriate "not found" message
for key in sections:
    if sections[key] == "Non trouvée dans le CV":
        sections[key] = not_found

# Print results
print(f"\nLangue détectée : {language}\n")
print(f"=== {headers['experience']} ===")
print(sections["experience"][:800] + "\n")
print(f"=== {headers['education']} ===")
print(sections["education"][:500] + "\n")
print(f"=== {headers['skills']} ===")
print(sections["skills"][:300] + "\n")
print(f"=== {headers['academic_projects']} ===")
print(sections["academic_projects"][:300] + "\n")


Langue détectée : English

=== EXPERIENCE ===
OUMEYMA TIBAOUI oumeimatibaoui gmail co m 2023 2024 ELITEZCOM Nabeul Developed a web application for invoicing and commercial operations using Angular frontend and Symfony backend Integrated an AI powered module using Python for generating data driven business insights Built a smart virtual assistant that interacts with the database in real time for data access and updates

=== EDUCATION ===
PROGRAMMING LANGUAGES FRAMEWORKS Python Java C PHP JavaScript TypeScript Dart Angular Symfony Spring Boot Flutter Bootstrap DATABASES FireBase MySQL SQL MongoDB SOFT

=== SKILLS ===
Teamwork Communication Time Management Problem Solving Organization PROJETS ACADEMIQUES LinkedIn FINAL YEAR INTERN ONLINE BILLING AND COMMERCIAL MANAGEMENT SYSTEM TECHNICAL SKILLS Arabic Native French Intermediate English Intermediate Turkish Beginner LANGUAGES 2023 ELITEZCOM Nabeul Designed and dev

=== ACADEMIC PROJECTS ===
Not found in the CV



In [5]:
pip install spacy

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip uninstall numpy

In [ ]:
pip install numpy

In [ ]:
import re
import fitz
import spacy
from langdetect import detect
import io

def extract_text(resume, ext):
    """Extract text from a PDF file."""
    if ext.lower() != ".pdf":
        raise ValueError("Only PDF files are supported")
    
    text = ""
    try:
        if isinstance(resume, io.BytesIO):
            doc = fitz.open(stream=resume, filetype="pdf")
        else:
            doc = fitz.open(resume)
        
        for page in doc:
            page_text = page.get_text()
            if page_text.strip():
                text += page_text + "\n"
        doc.close()
    except Exception as e:
        raise Exception(f"Failed to extract text: {e}")
    return text

def detect_language(text):
    """Detect if the text is English or French."""
    try:
        lang = detect(text[:500])
        return "French" if lang == "fr" else "English" if lang == "en" else f"Other ({lang})"
    except:
        return "Unknown"

def clean_text(text):
    """Clean text by removing special chars and normalizing spaces."""
    # Replace accented characters
    text = text.replace('à', 'a').replace('â', 'a').replace('ä', 'a')
    text = text.replace('é', 'e').replace('è', 'e').replace('ê', 'e').replace('ë', 'e')
    text = text.replace('î', 'i').replace('ï', 'i')
    text = text.replace('ô', 'o').replace('ö', 'o')
    text = text.replace('ù', 'u').replace('û', 'u').replace('ü', 'u')
    text = text.replace('ç', 'c')
    # Remove special characters, keep alphanumeric and spaces
    text = re.sub(r'[^A-Za-z0-9\s]', ' ', text)
    # Normalize spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def extract_name(nlp_doc, matcher):
    """Extract name using spaCy NER."""
    for ent in nlp_doc.ents:
        if ent.label_ == "PERSON":
            return ent.text
    return None

def extract_email(text):
    """Extract email address using regex."""
    match = re.search(r'[\w\.-]+@[\w\.-]+', text)
    return match.group(0) if match else None

def extract_mobile_number(text, custom_regex=None):
    """Extract phone number using regex."""
    if custom_regex:
        match = re.search(custom_regex, text)
    else:
        match = re.search(r'(\+?\d{2,4}[-.\s]?)?(\(?\d{2,4}\)?[-.\s]?)?(\d{2,4}[-.\s]?){2,4}', text)
    return match.group(0) if match else None

def extract_sections_regex(raw_text, language):
    """Extract Experience, Education, Skills, and Projects using regex."""
    sections = {
        "experience": None,
        "education": None,
        "skills": None,
        "academic_projects": None
    }

    # Define patterns based on language
    if language == "French":
        exp_pattern = r'\b(exp[ée]rience|emploi|exp[ée]rience professionnelle)\b\s*([:;\-\n#|])(.*?)(?=\b(formation|[ée]ducation|comp[ée]tences|comp[ée]tences techniques|projets acad[ée]miques|projets)\b|$)'
        edu_pattern = r'\b(formation|[ée]ducation)\b\s*([:;\-\n#|])(.*?)(?=\b(comp[ée]tences|comp[ée]tences techniques|exp[ée]rience|emploi|exp[ée]rience professionnelle|projets acad[ée]miques|projets)\b|$)'
        skills_pattern = r'\b(comp[ée]tences|comp[ée]tences techniques)\b\s*([:;\-\n#|])(.*?)(?=\b(formation|[ée]ducation|exp[ée]rience|emploi|exp[ée]rience professionnelle|projets acad[ée]miques|projets)\b|$)'
        projects_pattern = r'\b(projets acad[ée]miques|projets)\b\s*([:;\-\n#|])(.*?)(?=\b(formation|[ée]ducation|comp[ée]tences|comp[ée]tences techniques|exp[ée]rience|emploi|exp[ée]rience professionnelle)\b|$)'
        not_found = "Non trouvée dans le CV"
    else:
        exp_pattern = r'\b(experience|work history|professional experience)\b\s*([:;\-\n#|])(.*?)(?=\b(education|academic background|skills|technical skills|academic projects|projects)\b|$)'
        edu_pattern = r'\b(education|academic background)\b\s*([:;\-\n#|])(.*?)(?=\b(skills|technical skills|experience|work history|professional experience|academic projects|projects)\b|$)'
        skills_pattern = r'\b(skills|technical skills)\b\s*([:;\-\n#|])(.*?)(?=\b(education|academic background|experience|work history|professional experience|academic projects|projects)\b|$)'
        projects_pattern = r'\b(academic projects|projects)\b\s*([:;\-\n#|])(.*?)(?=\b(education|academic background|skills|technical skills|experience|work history|professional experience)\b|$)'
        not_found = "Not found in the CV"

    # Extract sections
    exp_match = re.search(exp_pattern, raw_text, re.IGNORECASE | re.DOTALL)
    edu_match = re.search(edu_pattern, raw_text, re.IGNORECASE | re.DOTALL)
    skills_match = re.search(skills_pattern, raw_text, re.IGNORECASE | re.DOTALL)
    projects_match = re.search(projects_pattern, raw_text, re.IGNORECASE | re.DOTALL)

    # Assign cleaned text to sections
    if exp_match:
        sections["experience"] = clean_text(exp_match.group(3).strip())
    else:
        sections["experience"] = not_found
    if edu_match:
        sections["education"] = clean_text(edu_match.group(3).strip())
    else:
        sections["education"] = not_found
    if skills_match:
        sections["skills"] = clean_text(skills_match.group(3).strip())
    else:
        sections["skills"] = not_found
    if projects_match:
        sections["academic_projects"] = clean_text(projects_match.group(3).strip())
    else:
        sections["academic_projects"] = not_found

    return sections

def get_number_of_pages(resume, ext):
    """Get the number of pages in the PDF."""
    if ext.lower() != ".pdf":
        return None
    try:
        if isinstance(resume, io.BytesIO):
            doc = fitz.open(stream=resume, filetype="pdf")
        else:
            doc = fitz.open(resume)
        pages = doc.page_count
        doc.close()
        return pages
    except:
        return None

In [ ]:
pip install flair

In [ ]:
from flair.data import Sentence
from flair.embeddings import TransformerDocumentEmbeddings
from flair.similarity import DocumentSimilarity

# Mock setup (replace with actual flair initialization)
# embedding = TransformerDocumentEmbeddings('distilbert-base-uncased')
# similarity = DocumentSimilarity(embedding)

def mock_flair_similarity(text1, text2):
    # Placeholder for flair similarity; use keyword overlap as a proxy
    standard_keywords = ["worked", "developed", "experience", "team", "project"]
    text1_words = text1.lower().split()
    text2_words = text2.lower().split()
    common_words = set(text1_words) & set(text2_words) & set(standard_keywords)
    return len(common_words) / len(standard_keywords)  # Normalized score [0, 1]

def find_best_experience_block(lines, standard_experience):
    if len(lines) < 5:
        return "CV too short for 5-line window."

    # Standard experience description
    standard_desc = standard_experience.lower()

    # Initialize sliding window
    window_size = 5
    best_start = 0
    best_end = window_size
    max_score = 0.0
    current_start = 0

    # Slide 5-line window across CV
    while current_start <= len(lines) - window_size:
        window = "\n".join(lines[current_start:current_start + window_size])
        score = mock_flair_similarity(window, standard_desc)
        if score > max_score:
            max_score = score
            best_start = current_start
            best_end = current_start + window_size
        current_start += 1

    # Optimize window size around best 5-line block
    best_block = "\n".join(lines[best_start:best_end])
    current_block = best_block
    current_score = max_score
    min_lines = 1
    max_lines = len(lines)

    # Expand or contract to find optimal length
    while True:
        # Try adding a line
        if best_end < len(lines):
            new_end = best_end + 1
            new_block = "\n".join(lines[best_start:new_end])
            new_score = mock_flair_similarity(new_block, standard_desc)
            if new_score > current_score:
                current_block = new_block
                current_score = new_score
                best_end = new_end
            else:
                break
        # Try removing a line from start if more than 5 lines
        elif best_start < best_end - 1:
            new_start = best_start + 1
            new_block = "\n".join(lines[new_start:best_end])
            new_score = mock_flair_similarity(new_block, standard_desc)
            if new_score >= current_score:
                current_block = new_block
                current_score = new_score
                best_start = new_start
            else:
                break
        else:
            break

    return current_block, best_end - best_start

# Example usage
cv_text = """Passionate and dedicated software engineer with a strong foundation in computer science. I enjoy creating innovative and practical
solutions across desktop, web, and mobile platforms. I am eager to learn, proactive in my approach, and constantly seeking new challenges
where I can apply both my technical and soft skills to contribute meaningfully to any team or project.
+216 23 577 840
Tunisian | Dar Chaaben El Fehri, Nabeul
WORK EXPERIENCE
OUMEYMA TIBAOUI
oumeimatibaoui@gmail.co
m
2023–2024 | ELITEZCOM, Nabeul
• Developed a web application for invoicing and commercial operations using Angular (frontend) and Symfony (backend).
• Integrated an AI-powered module using Python for generating data-driven business insights.
• Built a smart virtual assistant that interacts with the database in real-time for data access and updates.
EDUCATION
PROGRAMMING LANGUAGES & FRAMEWORKS:
 • Python – Java – C++ – PHP – JavaScript – TypeScript – Dart"""
lines = cv_text.splitlines()
standard_experience = "I worked as a software engineer developing web applications and collaborated with teams."
best_block, line_count = find_best_experience_block(lines, standard_experience)
print(f"Best experience block:\n{best_block}")
print(f"Number of lines: {line_count}")

In [ ]:
print("hello!")

In [1]:
pip install flair

In [2]:
import re
import fitz  # PyMuPDF
from langdetect import detect
from flair.embeddings import TransformerDocumentEmbeddings
from flair.data import Sentence

PDF_PATH = "C:/Users/OUMAIMA/Downloads/cv_oumeymaFinal (1).pdf"
# Charger un modèle transformer de Flair (comme BERT)
embedding_model = TransformerDocumentEmbeddings('bert-base-uncased')

def extract_text_from_pdf(pdf_path):
    """Extract text from PDF using PyMuPDF."""
    text = ""
    doc = fitz.open(pdf_path)
    for page in doc:
        page_text = page.get_text()
        if page_text.strip():
            text += page_text + "\n"
    doc.close()
    return text

def clean_text(text):
    """Clean text by replacing accented characters and removing special chars except \n and spaces."""
    text = text.replace('à', 'a').replace('â', 'a').replace('ä', 'a')
    text = text.replace('é', 'e').replace('è', 'e').replace('ê', 'e').replace('ë', 'e')
    text = text.replace('î', 'i').replace('ï', 'i')
    text = text.replace('ô', 'o').replace('ö', 'o')
    text = text.replace('ù', 'u').replace('û', 'u').replace('ü', 'u')
    text = text.replace('ç', 'c')
    text = re.sub(r'[^\w\s\n]', '', text)
    return text

def detect_language(text):
    """Detect if the text is English or French with a larger sample and header check."""
    try:
        sample = text[:1000].lower()
        lang = detect(sample)
        if re.search(r'\b(formation|éducation|compétences|projets académiques)\b', sample):
            return "French"
        return "French" if lang == "fr" else "English" if lang == "en" else f"Other ({lang})"
    except:
        return "Unknown"

def mock_flair_similarity(text1, text2):
    """Placeholder for flair similarity; use keyword overlap as a proxy with section-specific keywords."""
    # Define keywords for each section type
    section_keywords = {
        "experience": ["worked", "developed", "experience", "team", "project", "intern", "built"],
        "skills": ["skills", "languages", "frameworks", "proficient", "tools", "databases"],
        "education": ["education", "degree", "graduated", "school", "university", "engineering"]
    }
    # Use the appropriate keyword set based on section_type
    keywords = section_keywords.get(section_type, section_keywords["experience"])
    text1_words = text1.lower().split()
    text2_words = text2.lower().split()
    common_words = set(text1_words) & set(text2_words) & set(keywords)
    score = len(common_words) / len(keywords) if keywords else 0.0  # Normalized score [0, 1]
    print(f"Text1 words: {text1_words[:5]}... Text2 words: {text2_words[:5]}... Common: {common_words} (Keywords: {keywords})")
    return score
def flair_similarity(text1, text2):
    """Compare deux textes en utilisant des embeddings contextuels Flair."""
    sentence1 = Sentence(text1)
    sentence2 = Sentence(text2)

    # Générer les embeddings
    embedding_model.embed(sentence1)
    embedding_model.embed(sentence2)

    # Extraire les vecteurs
    vec1 = sentence1.embedding.detach().numpy()
    vec2 = sentence2.embedding.detach().numpy()

    # Cosine similarity entre les vecteurs
    score = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

    return float(score)  # score entre -1 et 1
def find_best_section_block(lines, standard_description, section_type="experience"):
    """Find the best block for a given section (experience, skills, or education) by sliding a 5-line window and optimizing."""
    if len(lines) < 5:
        return "CV too short for 5-line window."

    # Standard description for the specific section
    standard_desc = standard_description.lower()

    # Initialize sliding window
    window_size = 5
    best_start = 0
    best_end = window_size
    max_score = 0.0
    current_start = 0

    # Slide 5-line window across CV with debugging
    print(f"\n=== SLIDING WINDOW PROCESS ({section_type.capitalize()}) ===")
    while current_start <= len(lines) - window_size:
        window = "\n".join(lines[current_start:current_start + window_size])
        score = flair_similarity(window, standard_desc)
        print(f"Window {current_start}-{current_start + window_size}: {window[:50]}... (Score: {score:.2f})")
        if score > max_score:
            max_score = score
            best_start = current_start
            best_end = current_start + window_size
            print(f"  -> New best window found! Score: {max_score:.2f}")
        current_start += 1

    # Optimize window size around best 5-line block
    best_block = "\n".join(lines[best_start:best_end])
    current_block = best_block
    current_score = max_score
    min_lines = 1
    max_lines = len(lines)

    print(f"\n=== WINDOW OPTIMIZATION ({section_type.capitalize()}) ===")
    print(f"Initial best block ({best_end - best_start} lines):\n{best_block}")

    # Expand or contract to find optimal length
    while True:
        # Try adding a line
        if best_end < len(lines):
            new_end = best_end + 1
            new_block = "\n".join(lines[best_start:new_end])
            new_score = flair_similarity(new_block, standard_desc)
            print(f"Trying +1 line ({new_end - best_start} lines): {new_block[:50]}... (Score: {new_score:.2f})")
            if new_score > current_score:
                current_block = new_block
                current_score = new_score
                best_end = new_end
                print(f"  -> Improved! New score: {current_score:.2f}")
            else:
                print(f"  -> No improvement, stopping expansion.")
                break
        # Try removing a line from start if more than 5 lines
        elif best_start < best_end - 1:
            new_start = best_start + 1
            new_block = "\n".join(lines[new_start:best_end])
            new_score = flair_similarity(new_block, standard_desc)
            print(f"Trying -1 line ({best_end - new_start} lines): {new_block[:50]}... (Score: {new_score:.2f})")
            if new_score >= current_score:
                current_block = new_block
                current_score = new_score
                best_start = new_start
                print(f"  -> Improved! New score: {current_score:.2f}")
            else:
                print(f"  -> No improvement, stopping contraction.")
                break
        else:
            break

    return current_block, best_end - best_start

# MAIN
raw_text = extract_text_from_pdf(PDF_PATH)
cleaned_text = clean_text(raw_text)
lines = cleaned_text.splitlines()  # Split into lines here
language = detect_language(raw_text)
print(f"\n📄 Langue détectée : {language}")

# Standard descriptions for each section
standard_experience = "Completed a 4-month internship at TechSolutions where I developed a RESTful API using Node.js and MongoDB, improving data processing."
standard_skills = "Proficient in Python, Java, JavaScript, and frameworks like Angular, with experience in databases such as MySQL and Firebase."
standard_education = "Graduated with a Bachelor’s Degree in Computer Science from a university in 2023."

# Detect blocks for each section
best_experience_block, experience_line_count = find_best_section_block(lines, standard_experience, "experience")
best_skills_block, skills_line_count = find_best_section_block(lines, standard_skills, "skills")
best_education_block, education_line_count = find_best_section_block(lines, standard_education, "education")

# Print results
print("\n=== SECTION D'EXPÉRIENCE DÉTECTÉE ===")
print(f"Best experience block:\n{best_experience_block}")
print(f"Number of lines: {experience_line_count}")

print("\n=== SECTION DES COMPÉTENCES DÉTECTÉE ===")
print(f"Best skills block:\n{best_skills_block}")
print(f"Number of lines: {skills_line_count}")

print("\n=== SECTION D'ÉDUCATION DÉTECTÉE ===")
print(f"Best education block:\n{best_education_block}")
print(f"Number of lines: {education_line_count}")

ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.11 from "C:\Users\OUMAIMA\anaconda3\python.exe"
  * The NumPy version is: "1.24.3"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: DLL load failed while importing _multiarray_umath: Le module spécifié est introuvable.
